# Mod 2 Project Cleaning Notebook
### Amir Edris, Ben Inoyatov
---

In [1]:
# importing the standard libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
import pickle as pkl
plt.style.use('fivethirtyeight')


pd.set_option('display.max_columns', 500)

In [2]:
# math m and portugese p classes. Datasets taken from the UCI Machine Learning Repository 
p_orig = pd.read_csv('student_por.csv', sep=';')
m_orig = pd.read_csv('student_mat.csv', sep=';')

In [3]:
p_orig.head()

# many nominal/binary features ie Mothers Education (Medu) 1-5 for different degree levels etc

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [4]:
# since our columns are the same for both the math and Portugese dfs, we'll make one function 
def clean_up(data):
    '''
    cleanup the dfs, using dummy variables for binary columns
    '''
    data.address = data.address.apply(lambda x: 'Urban' if x == 'U' else 'Rural')
    data = pd.get_dummies(data=data, columns=['address'], prefix='address_type', drop_first=True)
    data.famsize = data.famsize.apply(lambda x: 'BIG_FAMILY' if x == 'GT3' else 'SMALL_FAMILY') 
    data = pd.get_dummies(data=data, columns=['famsize'])
    data = data.drop(columns='famsize_SMALL_FAMILY')
    data.Pstatus = data.Pstatus.apply(lambda x: 'Together' if x == 'T' else 'Apart') 
    data = pd.get_dummies(data=data, columns=['Pstatus'], drop_first=True)
    data = pd.get_dummies(data=data, columns=['Mjob', 'Fjob'])
    data = data.drop(columns=['Mjob_other', 'Fjob_other'])
    data = pd.get_dummies(data=data, columns=['reason'])
    data = data.drop(columns=['reason_other'])
    data.schoolsup = data.schoolsup.apply(lambda x: 1 if x == 'yes' else 0) 
    data.famsup = data.famsup.apply(lambda x: 1 if x == 'yes' else 0) 
    data['tutor'] = data.paid.apply(lambda x: 1 if x == 'yes' else 0)
    data = data.drop(columns='paid')
    data.activities = data.activities.apply(lambda x: 1 if x == 'yes' else 0) 
    data.nursery = data.nursery.apply(lambda x: 1 if x == 'yes' else 0) 
    data.higher = data.higher.apply(lambda x: 1 if x == 'yes' else 0) 
    data.internet = data.internet.apply(lambda x: 1 if x == 'yes' else 0) 
    data.romantic = data.romantic.apply(lambda x: 1 if x == 'yes' else 0) 
    return data 

In [5]:
p = clean_up(p_orig)
m = clean_up(m_orig)

In [6]:
p.head()

,school,sex,age,Medu,Fedu,guardian,traveltime,studytime,failures,schoolsup,famsup,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,address_type_Urban,famsize_BIG_FAMILY,Pstatus_Together,Mjob_at_home,Mjob_health,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_reputation,tutor
0,GP,F,18,4,4,mother,2,2,0,1,0,0,1,1,0,0,4,3,4,1,1,3,4,0,11,11,1,1,0,1,0,0,0,0,0,0,1,1,0,0,0
1,GP,F,17,1,1,father,1,2,0,0,1,0,0,1,1,0,5,3,3,1,1,3,2,9,11,11,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0
2,GP,F,15,1,1,mother,1,2,0,1,0,0,1,1,1,0,4,3,2,2,3,3,6,12,13,12,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0
3,GP,F,15,4,2,mother,1,3,0,0,1,1,1,1,1,1,3,2,2,1,1,5,0,14,14,14,1,1,1,0,1,0,0,0,0,1,0,0,1,0,0
4,GP,F,16,3,3,father,1,2,0,0,1,0,1,1,0,0,4,3,2,1,2,5,0,11,13,13,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0


In [7]:
m.head()

,school,sex,age,Medu,Fedu,guardian,traveltime,studytime,failures,schoolsup,famsup,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,address_type_Urban,famsize_BIG_FAMILY,Pstatus_Together,Mjob_at_home,Mjob_health,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_reputation,tutor
0,GP,F,18,4,4,mother,2,2,0,1,0,0,1,1,0,0,4,3,4,1,1,3,6,5,6,6,1,1,0,1,0,0,0,0,0,0,1,1,0,0,0
1,GP,F,17,1,1,father,1,2,0,0,1,0,0,1,1,0,5,3,3,1,1,3,4,5,5,6,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0
2,GP,F,15,1,1,mother,1,2,3,1,0,0,1,1,1,0,4,3,2,2,3,3,10,7,8,10,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1
3,GP,F,15,4,2,mother,1,3,0,0,1,1,1,1,1,1,3,2,2,1,1,5,2,15,14,15,1,1,1,0,1,0,0,0,0,1,0,0,1,0,1
4,GP,F,16,3,3,father,1,2,0,0,1,0,1,1,0,0,4,3,2,1,2,5,4,6,10,10,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1


### Feature Selection
- G1 (first trimester grade) and G2 (second trimester grade) are both correlated with each other and to G3, our target variable. The purpose of these tests was to see which factors led to a student acheiving a higher final score so we did not include the previous grades in our model. 
    - If you excel at math or excel at language arts, chances are you'll recieve high marks all year 
- We also did not want our model to account for the student's age and sex. 
- Our target is G3, the final grade of the school year. 

In [8]:
features = [x for x in p.columns if (x != 'age') and x != 'G1' and x != 'G2' and x!='G3' and x!= 'sex']
features

['school',
 'Medu',
 'Fedu',
 'guardian',
 'traveltime',
 'studytime',
 'failures',
 'schoolsup',
 'famsup',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'address_type_Urban',
 'famsize_BIG_FAMILY',
 'Pstatus_Together',
 'Mjob_at_home',
 'Mjob_health',
 'Mjob_services',
 'Mjob_teacher',
 'Fjob_at_home',
 'Fjob_health',
 'Fjob_services',
 'Fjob_teacher',
 'reason_course',
 'reason_home',
 'reason_reputation',
 'tutor']

In [9]:
len(features)

36

In [10]:
# new df with these features for the two classes, prepping our test feature data set
p_feature = p[features]
m_feature = m[features]

### Feature engineering 
- Our test "super" feature 'TestCol' was an aggregation of values that may indicate your drive for education, family support to education, and social events like drinking or going out 
- We scaled this column to be 0-5

In [11]:
p_feature

,school,Medu,Fedu,guardian,traveltime,studytime,failures,schoolsup,famsup,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,address_type_Urban,famsize_BIG_FAMILY,Pstatus_Together,Mjob_at_home,Mjob_health,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_reputation,tutor
0,GP,4,4,mother,2,2,0,1,0,0,1,1,0,0,4,3,4,1,1,3,4,1,1,0,1,0,0,0,0,0,0,1,1,0,0,0
1,GP,1,1,father,1,2,0,0,1,0,0,1,1,0,5,3,3,1,1,3,2,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0
2,GP,1,1,mother,1,2,0,1,0,0,1,1,1,0,4,3,2,2,3,3,6,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0
3,GP,4,2,mother,1,3,0,0,1,1,1,1,1,1,3,2,2,1,1,5,0,1,1,1,0,1,0,0,0,0,1,0,0,1,0,0
4,GP,3,3,father,1,2,0,0,1,0,1,1,0,0,4,3,2,1,2,5,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,2,3,mother,1,3,1,0,0,1,0,1,1,0,5,4,2,1,2,5,4,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0
645,MS,3,1,mother,1,2,0,0,1,0,1,1,1,0,4,3,4,1,1,1,4,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0
646,MS,1,1,mother,2,2,0,0,0,1,1,1,0,0,1,1,1,1,1,5,6,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0
647,MS,3,1,mother,2,1,0,0,0,0,0,1,1,0,2,4,5,3,4,2,6,1,0,1,0,0,1,0,0,0,1,0,1,0,0,0


In [12]:
# little extra cleaning, we decided to keep these columns
p_feature = pd.get_dummies(p_feature, columns=['guardian'], prefix='guardian')
p_feature.drop(columns=['guardian_other'], inplace=True)
p_feature = pd.get_dummies(p_feature, columns=['school'], drop_first=True, prefix='school')

m_feature = pd.get_dummies(m_feature, columns=['guardian'], prefix='guardian')
m_feature.drop(columns=['guardian_other'], inplace=True)
m_feature = pd.get_dummies(m_feature, columns=['school'], drop_first=True, prefix='school')




In [13]:
m_feature.head()

,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,address_type_Urban,famsize_BIG_FAMILY,Pstatus_Together,Mjob_at_home,Mjob_health,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_reputation,tutor,guardian_father,guardian_mother,school_MS
0,4,4,2,2,0,1,0,0,1,1,0,0,4,3,4,1,1,3,6,1,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0
1,1,1,1,2,0,0,1,0,0,1,1,0,5,3,3,1,1,3,4,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0
2,1,1,1,2,3,1,0,0,1,1,1,0,4,3,2,2,3,3,10,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,4,2,1,3,0,0,1,1,1,1,1,1,3,2,2,1,1,5,2,1,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0
4,3,3,1,2,0,0,1,0,1,1,0,0,4,3,2,1,2,5,4,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0


In [14]:
p_feature['super'] =  (((5/4)*p_feature.Medu)+((5/4)*p_feature.Fedu)+(5*p_feature.famsup)+(p_feature.famrel)+(p_feature.health)+(5*p_feature.activities)+(5*p_feature.higher)-p_feature.Dalc-p_feature.Walc-p_feature.goout)
m_feature['super'] =  (((5/4)*m_feature.Medu)+((5/4)*m_feature.Fedu)+(5*m_feature.famsup)+(m_feature.famrel)+(m_feature.health)+(5*m_feature.activities)+(5*m_feature.higher)-m_feature.Dalc-m_feature.Walc-m_feature.goout)


In [15]:
m_orig.to_csv('MathClassOriginal.csv')
p_orig.to_csv('PortugeseClassOriginal.csv')
m.to_csv('MathClassCleaned.csv')
p.to_csv('PortugeseClassCleaned.csv')

In [16]:
p_feature.to_csv('p_Features.csv')
m_feature.to_csv('m_Features.csv')

In [17]:
pkl.dump(features,open('features.pkl','wb'))

In [18]:
pkl.dump(p_feature,open('p_feature.pkl','wb'))

In [19]:
pkl.dump(m_feature,open('m_feature.pkl','wb'))